In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFECV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline
from scipy.stats import randint, uniform
import warnings
warnings.filterwarnings('ignore')



In [ ]:
# Чтение файла CSV
df = pd.read_excel('data/radiomics.xlsx', dtype=str, sheet_name='T2')
df.dropna(thresh=50, inplace=True)
df.drop(['№а/к', 'ФИО', 'пол','Feature Name','PyRadiomics',  'Numpy', 'SimpleITK', 'PyWavelet', 'Python', 'Settings','EnabledImageTypes','Hash', 'Dimensionality', 'Spacing', 'Size','Mean', 'Minimum', 'Maximum' ,'Hash.1', 'Spacing.1', 'Size.1','BoundingBox','VoxelNum', 'VolumeNum', 'CenterOfMassIndex', 'CenterOfMass' ], axis=1, inplace=True)
df.rename(columns={'Unnamed: 1':'Diagnosis'}, inplace=True)
df['Diagnosis'] = df['Diagnosis'].str.replace(r'^C.*', '1', regex=True)
df['Diagnosis'] = df['Diagnosis'].str.replace(r'^D.*', '0', regex=True)
df['Diagnosis']= df['Diagnosis'].astype(int)

In [39]:
# df.to_excel('df_with_index.xlsx', index=True)  # Сохранение с индексами

In [48]:
# Выбираем все колонки, кроме Diagnosis
cols_to_process = [col for col in df.columns if col != 'Diagnosis']

# Замена точек на запятые во всех ячейках выбранных колонок
df[cols_to_process] = df[cols_to_process].applymap(
    lambda x: str(x).replace(',', '.') if pd.notna(x) else x
)

# Принудительное приведение к типу float
df[cols_to_process] = df[cols_to_process].astype(float)

In [ ]:
# Загрузка данных (замените на ваш путь к файлу)
# df = pd.read_excel('your_file.xlsx')

# Предполагаем, что df уже загружен
print("Размер данных:", df.shape)
print("\nПервые 5 строк:")
print(df.head())

# Проверка на пропущенные значения
print("\nПропущенные значения:")
print(df.isnull().sum())

# Удаление строк с пропущенными значениями или заполнение средним
df = df.dropna()  # или df.fillna(df.mean(), inplace=True)


In [ ]:
# Разделение на признаки и целевую переменную
X = df.drop('Diagnosis', axis=1)
y = df['Diagnosis']

# Проверка баланса классов
print("\nРаспределение классов:")
print(y.value_counts())
print("Процент больных:", (y.sum() / len(y)) * 100)

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
X_test.head()

In [51]:
# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:

# 1. Анализ важности признаков с помощью статистических методов
def statistical_feature_analysis(X, y):
    """Статистический анализ важности признаков"""
    selector = SelectKBest(score_func=f_classif, k='all')
    selector.fit(X, y)
    
    # Создаем DataFrame с результатами
    feature_scores = pd.DataFrame({
        'Feature': X.columns,
        'F_Score': selector.scores_,
        'P_Value': selector.pvalues_
    })
    
    feature_scores = feature_scores.sort_values('F_Score', ascending=False)
    return feature_scores

# 2. Функция для обучения и оценки моделей
def evaluate_models(X_train, X_test, y_train, y_test):
    """Оценка различных классификаторов"""
    models = {
        'Logistic Regression': LogisticRegression(random_state=42),
        'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
        'Support Vector Classifier': SVC(random_state=42, probability=True),
        'Random Forest': RandomForestClassifier(random_state=42),
        'Gaussian Naive Bayes': GaussianNB(),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(random_state=42)
    }
    
    results = []
    
    for name, model in models.items():
        # Обучение модели
        model.fit(X_train, y_train)
        
        # Предсказания
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Метрики
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        
        roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
        
        results.append({
            'Model': name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'ROC-AUC': roc_auc
        })
        
        print(f"{name}: Accuracy = {accuracy:.3f}, F1 = {f1:.3f}")
    
    return pd.DataFrame(results)
# 3. Функция для подбора гиперпараметров
def hyperparameter_tuning(X_train, y_train):
    """Подбор гиперпараметров для лучших моделей"""
    
    # Настройка Random Forest
    rf_param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    }
    
    rf_search = RandomizedSearchCV(
        RandomForestClassifier(random_state=42),
        rf_param_grid,
        n_iter=50,
        cv=5,
        scoring='f1',
        random_state=42,
        n_jobs=-1
    )
    rf_search.fit(X_train, y_train)
    
    # Настройка SVC
    svc_param_grid = {
        'C': uniform(0.1, 10),
        'gamma': uniform(0.01, 1),
        'kernel': ['rbf', 'linear']
    }
    
    svc_search = RandomizedSearchCV(
        SVC(probability=True, random_state=42),
        svc_param_grid,
        n_iter=50,
        cv=5,
        scoring='f1',
        random_state=42,
        n_jobs=-1
    )
    svc_search.fit(X_train, y_train)
    
    # Настройка Logistic Regression
    lr_param_grid = {
        'C': uniform(0.1, 10),
        'penalty': ['l2', 'none'],
        'solver': ['lbfgs', 'newton-cg']
    }
    
    lr_search = RandomizedSearchCV(
        LogisticRegression(random_state=42),
        lr_param_grid,
        n_iter=30,
        cv=5,
        scoring='f1',
        random_state=42,
        n_jobs=-1
    )
    lr_search.fit(X_train, y_train)
    
    return {
        'Random Forest': rf_search.best_estimator_,
        'SVC': svc_search.best_estimator_,
        'Logistic Regression': lr_search.best_estimator_
    }

# 4. Анализ важности признаков из лучших моделей
def analyze_feature_importance(models, feature_names):
    """Анализ важности признаков из обученных моделей"""
    importance_results = {}
    
    for name, model in models.items():
        if hasattr(model, 'feature_importances_'):
            # Для моделей с feature_importances_
            importances = model.feature_importances_
            indices = np.argsort(importances)[::-1]
            
            importance_df = pd.DataFrame({
                'Feature': feature_names[indices],
                'Importance': importances[indices]
            })
            importance_results[name] = importance_df
            
        elif hasattr(model, 'coef_'):
            # Для линейных моделей с коэффициентами
            if len(model.coef_.shape) > 1:
                coef = model.coef_[0]  # для многоклассовой классификации
            else:
                coef = model.coef_
            
            indices = np.argsort(np.abs(coef))[::-1]
            
            importance_df = pd.DataFrame({
                'Feature': feature_names[indices],
                'Coefficient': coef[indices],
                'Absolute_Coefficient': np.abs(coef[indices])
            })
            importance_results[name] = importance_df
    
    return importance_results

# Основной процесс анализа
print("=" * 50)
print("СТАТИСТИЧЕСКИЙ АНАЛИЗ ПРИЗНАКОВ")
print("=" * 50)

# Статистический анализ
stat_results = statistical_feature_analysis(X, y)
print(stat_results.head(10))

print("\n" + "=" * 50)
print("ОЦЕНКА БАЗОВЫХ МОДЕЛЕЙ")
print("=" * 50)

# Оценка базовых моделей
base_results = evaluate_models(X_train_scaled, X_test_scaled, y_train, y_test)

print("\n" + "=" * 50)
print("ПОДБОР ГИПЕРПАРАМЕТРОВ ДЛЯ ЛУЧШИХ МОДЕЛЕЙ")
print("=" * 50)

# Подбор гиперпараметров
best_models = hyperparameter_tuning(X_train_scaled, y_train)

# Оценка настроенных моделей
tuned_results = []
for name, model in best_models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    tuned_results.append({
        'Model': f'Tuned {name}',
        'Accuracy': accuracy,
        'F1-Score': f1
    })
    
    print(f"Tuned {name}: Accuracy = {accuracy:.3f}, F1 = {f1:.3f}")

print("\n" + "=" * 50)
print("АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ")
print("=" * 50)
# Анализ важности признаков
feature_importance = analyze_feature_importance(best_models, X.columns.values)

# Вывод наиболее важных признаков
for model_name, importance_df in feature_importance.items():
    print(f"\nТоп-10 признаков по модели {model_name}:")
    print(importance_df.head(10))

# 5. Визуализация результатов
def plot_results(stat_results, feature_importance):
    """Визуализация результатов анализа"""
    
    plt.figure(figsize=(15, 10))
    
    # Статистическая значимость
    plt.subplot(2, 2, 1)
    top_stats = stat_results.head(10)
    plt.barh(top_stats['Feature'], top_stats['F_Score'])
    plt.xlabel('F-Score')
    plt.title('Топ-10 признаков по F-Score')
    plt.gca().invert_yaxis()
    
    # Важность признаков из Random Forest
    plt.subplot(2, 2, 2)
    if 'Random Forest' in feature_importance:
        rf_importance = feature_importance['Random Forest'].head(10)
        plt.barh(rf_importance['Feature'], rf_importance['Importance'])
        plt.xlabel('Importance')
        plt.title('Топ-10 признаков (Random Forest)')
        plt.gca().invert_yaxis()
    
    # Коэффициенты Logistic Regression
    plt.subplot(2, 2, 3)
    if 'Logistic Regression' in feature_importance:
        lr_importance = feature_importance['Logistic Regression'].head(10)
        plt.barh(lr_importance['Feature'], lr_importance['Absolute_Coefficient'])
        plt.xlabel('Absolute Coefficient')
        plt.title('Топ-10 признаков (Logistic Regression)')
        plt.gca().invert_yaxis()
    
    # Сравнение моделей
    plt.subplot(2, 2, 4)
    all_results = pd.concat([base_results, pd.DataFrame(tuned_results)], ignore_index=True)
    models = all_results['Model']
    f1_scores = all_results['F1-Score']
    
    plt.barh(models, f1_scores)
    plt.xlabel('F1-Score')
    plt.title('Сравнение производительности моделей')
    plt.tight_layout()
    plt.show()

# Построение графиков
plot_results(stat_results, feature_importance)

# 6. Детальный анализ пороговых значений для лучших признаков
def analyze_thresholds(df, top_features, n_bins=10):
    """Анализ пороговых значений для топ-признаков"""
    threshold_analysis = {}
    
    for feature in top_features:
        # Создание бинов
        df['bin'] = pd.qcut(df[feature], n_bins, duplicates='drop')
        
        # Анализ заболеваемости по бинам
        bin_analysis = df.groupby('bin').agg({
            'Diagnosis': ['mean', 'count']
        }).round(3)
        
        bin_analysis.columns = ['Disease_Rate', 'Count']
        bin_analysis = bin_analysis.reset_index()
        
        threshold_analysis[feature] = bin_analysis
    
    return threshold_analysis

# Анализ порогов для топ-5 признаков
top_features = stat_results['Feature'].head(5).tolist()
threshold_results = analyze_thresholds(df, top_features)

print("\n" + "=" * 50)
print("АНАЛИЗ ПОРОГОВЫХ ЗНАЧЕНИЙ ДЛЯ ТОП-ПРИЗНАКОВ")
print("=" * 50)

for feature, analysis in threshold_results.items():
    print(f"\n{feature}:")
    print(analysis)

# 7. Финальный отчет
print("\n" + "=" * 50)
print("ФИНАЛЬНЫЙ ОТЧЕТ")
print("=" * 50)

# Определяем наиболее согласованные топ-признаки
all_important_features = set()

for importance_df in feature_importance.values():
    top_features = importance_df.head(5)['Feature'].tolist()
    all_important_features.update(top_features)

# Добавляем статистически значимые признаки
stat_top_features = stat_results.head(5)['Feature'].tolist()
all_important_features.update(stat_top_features)

print("Наиболее значимые признаки для диагностики:")
for i, feature in enumerate(all_important_features, 1):
    print(f"{i}. {feature}")

# Рекомендации по пороговым значениям
print("\nРекомендуется провести дополнительный анализ для определения оптимальных пороговых значений этих признаков.")